<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Stream Classification
---

## `NEWeather` dataset

**Description:** The National Oceanic and Atmospheric Administration (NOAA),
has compiled a database of weather measurements from over 7,000 weather 
stations worldwide. Records date back to the mid-1900s. Daily measurements
include a variety of features (temperature, pressure, wind speed, etc.) as
well as a series of indicators for precipitation and other weather-related
events. The `NEweather` dataset contains data from this database, specifically
from the Offutt Air Force Base in Bellevue, Nebraska ranging for over 50 years
(1949-1999).

**Features:** 8 Daily weather measurements
 
|       Attribute      | Description |
|:--------------------:|:-----------------------------|
| `temp`                   | Temperature
| `dew_pnt`                | Dew Point
| `sea_lvl_press`          | Sea Level Pressure
| `visibility`             | Visibility
| `avg_wind_spd`           | Average Wind Speed
| `max_sustained_wind_spd` | Maximum Sustained Wind Speed
| `max_temp`               | Maximum Temperature
| `min_temp`               | Minimum Temperature


**Class:** `rain` | 0: no rain, 1: rain
 
**Samples:** 18,159


In [1]:
import pandas as pd
from river.stream import iter_pandas
from river.metrics import Accuracy
from river.evaluate import progressive_val_score

In [2]:
data = pd.read_csv("../datasets/NEweather.csv")
features = data.columns[:-1]

In this example, we load the data from a csv file with `pandas.read_csv`, and we use the [iter_pandas](https://riverml.xyz/latest/api/stream/iter-pandas/) utility method to iterate over the `DataFrame`.

In [3]:
stream = iter_pandas(X=data[features], y=data['rain'])

## Naïve Bayes
---
[GaussianNB](https://riverml.xyz/latest/api/naive-bayes/GaussianNB/) maintains a Gaussian distribution $G_{cf}$ is maintained for each class $c$ and each feature $f$. Each Gaussian is updated using the amount associated with each feature; the details can be be found in proba.Gaussian. The joint log-likelihood is then obtained by summing the log probabilities of each feature associated with each class.

In [4]:
from river.naive_bayes import GaussianNB

model = GaussianNB()
metric = Accuracy()

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 71.87%
[2,000] Accuracy: 71.69%
[3,000] Accuracy: 71.96%
[4,000] Accuracy: 72.22%
[5,000] Accuracy: 72.35%
[6,000] Accuracy: 72.70%
[7,000] Accuracy: 73.32%
[8,000] Accuracy: 72.77%
[9,000] Accuracy: 72.65%
[10,000] Accuracy: 72.70%
[11,000] Accuracy: 73.07%
[12,000] Accuracy: 73.36%
[13,000] Accuracy: 73.07%
[14,000] Accuracy: 72.73%
[15,000] Accuracy: 72.90%
[16,000] Accuracy: 72.69%
[17,000] Accuracy: 72.56%
[18,000] Accuracy: 72.27%


Accuracy: 72.12%

## K-Nearest Neighbors
---
[KNN](https://riverml.xyz/latest/api/neighbors/KNNClassifier/) is a non-parametric classification method that keeps track of the last window_size training samples. The predicted class-label for a given query sample is obtained in two steps:

- Find the closest n_neighbors to the query sample in the data window. 
- Aggregate the class-labels of the n_neighbors to define the predicted class for the query sample.

In [5]:
from river.neighbors import KNNClassifier

model = KNNClassifier(n_neighbors=5, window_size=1000)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 76.38%
[2,000] Accuracy: 78.39%
[3,000] Accuracy: 79.03%
[4,000] Accuracy: 78.64%
[5,000] Accuracy: 78.24%
[6,000] Accuracy: 78.05%
[7,000] Accuracy: 78.14%
[8,000] Accuracy: 77.75%
[9,000] Accuracy: 77.76%
[10,000] Accuracy: 77.83%
[11,000] Accuracy: 77.93%
[12,000] Accuracy: 77.98%
[13,000] Accuracy: 77.85%
[14,000] Accuracy: 77.62%
[15,000] Accuracy: 77.71%
[16,000] Accuracy: 77.68%
[17,000] Accuracy: 77.53%
[18,000] Accuracy: 77.33%


Accuracy: 77.32%

## Hoeffding Tree
---

[Hoeffding Tree](https://riverml.xyz/latest/api/tree/HoeffdingTreeClassifier/) 

In [6]:
from river.tree import HoeffdingTreeClassifier

model = HoeffdingTreeClassifier()
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 71.57%
[2,000] Accuracy: 71.54%
[3,000] Accuracy: 71.86%
[4,000] Accuracy: 72.97%
[5,000] Accuracy: 73.13%
[6,000] Accuracy: 73.21%
[7,000] Accuracy: 73.88%
[8,000] Accuracy: 73.57%
[9,000] Accuracy: 73.70%
[10,000] Accuracy: 73.77%
[11,000] Accuracy: 74.15%
[12,000] Accuracy: 74.36%
[13,000] Accuracy: 74.53%
[14,000] Accuracy: 74.38%
[15,000] Accuracy: 74.57%
[16,000] Accuracy: 74.57%
[17,000] Accuracy: 74.46%
[18,000] Accuracy: 74.17%


Accuracy: 74.15%

Tree-based models are popular due to their interpretability. They use a tree data structure to model the data. When a sample arrives, it traverses the tree until it reaches a leaf node. Internal nodes define the path for a data sample based on the values of its features. Leaf nodes are models that provide predictions for unlabeled-samples and can update their internal state using the labels from labeled samples.

## Hoeffding Adaptive Tree
---
The [HAT](https://riverml.xyz/latest/api/tree/HoeffdingAdaptiveTreeClassifier/) model uses `ADWIN` to detect changes. If change is detected in a given branch, an alternate branch is created and eventually replaces the original branch if it shows better performance on new data.

In [8]:
from river.tree import HoeffdingAdaptiveTreeClassifier

model = HoeffdingAdaptiveTreeClassifier(seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metric, 
                      print_every=1000)

[1,000] Accuracy: 70.07%
[2,000] Accuracy: 70.64%
[3,000] Accuracy: 71.46%
[4,000] Accuracy: 72.97%
[5,000] Accuracy: 73.51%
[6,000] Accuracy: 73.80%
[7,000] Accuracy: 74.22%
[8,000] Accuracy: 73.78%
[9,000] Accuracy: 74.05%
[10,000] Accuracy: 74.13%
[11,000] Accuracy: 74.31%
[12,000] Accuracy: 74.38%
[13,000] Accuracy: 74.34%
[14,000] Accuracy: 74.11%
[15,000] Accuracy: 74.42%
[16,000] Accuracy: 74.49%
[17,000] Accuracy: 74.35%
[18,000] Accuracy: 73.90%


Accuracy: 73.91%

## Concept Drift Impact

Concept drift can negatively impact learning methods if not properly handled. Multiple real-world applications suffer **model degradation** as the models can not adapt to changes in the data.

---
## `AGRAWAL` dataset

We will load the data from a csv file. The data was generated using the `AGRAWAL` data generator with 3 **gradual drifts** at the 5k, 10k, and 15k marks. It contains 9 features, 6 numeric and 3 categorical.

There are 10 functions for generating binary class labels from the features. These functions determine whether a **loan** should be approved.

| Feature    | Description            | Values                                                                |
|------------|------------------------|-----------------------------------------------------------------------|
| `salary`     | salary                 | uniformly distributed from 20k to 150k                                |
| `commission` | commission             | if (salary <   75k) then 0 else uniformly distributed from 10k to 75k |
| `age`        | age                    | uniformly distributed from 20 to 80                                   |
| `elevel`     | education level        | uniformly chosen from 0 to 4                                          |
| `car`        | car maker              | uniformly chosen from 1 to 20                                         |
| `zipcode`    | zip code of the town   | uniformly chosen from 0 to 8                                          |
| `hvalue`     | value of the house     | uniformly distributed from 50k x zipcode to 100k x zipcode            |
| `hyears`     | years house owned      | uniformly distributed from 1 to 30                                    |
| `loan`       | total loan amount      | uniformly distributed from 0 to 500k                                  |

**Class:** `y` | 0: no loan, 1: loan
 
**Samples:** 20,000

`elevel`, `car`, and `zipcode` are categorical features.

In [10]:
data = pd.read_csv("../datasets/agr_a_20k.csv")
features = data.columns[:-1]

## Naïve Bayes

In [11]:
from river.naive_bayes import GaussianNB

model = GaussianNB()
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 84.68%
[2,000] Accuracy: 86.64%
[3,000] Accuracy: 87.20%
[4,000] Accuracy: 87.70%
[5,000] Accuracy: 87.54%
[6,000] Accuracy: 80.68%
[7,000] Accuracy: 74.84%
[8,000] Accuracy: 70.93%
[9,000] Accuracy: 68.03%
[10,000] Accuracy: 66.25%
[11,000] Accuracy: 66.71%
[12,000] Accuracy: 67.25%
[13,000] Accuracy: 67.91%
[14,000] Accuracy: 68.68%
[15,000] Accuracy: 69.24%
[16,000] Accuracy: 68.25%
[17,000] Accuracy: 67.40%
[18,000] Accuracy: 66.84%
[19,000] Accuracy: 66.25%
[20,000] Accuracy: 65.86%


Accuracy: 65.86%

## KNN with ADWIN
---

This classifier is an improvement from the regular kNN method, as it is resistant to concept drift. It uses the ADWIN change detector to decide which samples to keep and which ones to forget, and by doing so it regulates the sample window size.

In [12]:
from river.neighbors import KNNADWINClassifier
from river import compose

model = (
    compose.Discard('elevel', 'car', 'zipcode') |
    KNNADWINClassifier(n_neighbors=5, window_size=1000)
)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 55.86%
[2,000] Accuracy: 56.53%
[3,000] Accuracy: 57.19%
[4,000] Accuracy: 57.79%
[5,000] Accuracy: 57.41%
[6,000] Accuracy: 57.24%
[7,000] Accuracy: 58.88%
[8,000] Accuracy: 59.79%
[9,000] Accuracy: 60.75%
[10,000] Accuracy: 61.03%
[11,000] Accuracy: 60.02%
[12,000] Accuracy: 59.77%
[13,000] Accuracy: 59.70%
[14,000] Accuracy: 59.67%
[15,000] Accuracy: 59.40%
[16,000] Accuracy: 59.82%
[17,000] Accuracy: 61.01%
[18,000] Accuracy: 61.85%
[19,000] Accuracy: 62.63%
[20,000] Accuracy: 63.23%


Accuracy: 63.23%

## Hoeffding Tree

In [13]:
from river.tree import HoeffdingTreeClassifier

model = HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode'])
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 82.48%
[2,000] Accuracy: 82.89%
[3,000] Accuracy: 84.96%
[4,000] Accuracy: 86.50%
[5,000] Accuracy: 87.16%
[6,000] Accuracy: 80.85%
[7,000] Accuracy: 76.93%
[8,000] Accuracy: 74.75%
[9,000] Accuracy: 74.20%
[10,000] Accuracy: 74.49%
[11,000] Accuracy: 73.64%
[12,000] Accuracy: 73.57%
[13,000] Accuracy: 73.92%
[14,000] Accuracy: 74.65%
[15,000] Accuracy: 75.62%
[16,000] Accuracy: 74.25%
[17,000] Accuracy: 73.27%
[18,000] Accuracy: 72.83%
[19,000] Accuracy: 72.51%
[20,000] Accuracy: 72.37%


Accuracy: 72.37%

## Hoeffding Adaptive Tree

In [15]:
from river.tree import HoeffdingAdaptiveTreeClassifier

model = HoeffdingAdaptiveTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode'], seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metric, 
                      print_every=1000)

[1,000] Accuracy: 83.78%
[2,000] Accuracy: 87.54%
[3,000] Accuracy: 88.83%
[4,000] Accuracy: 90.15%
[5,000] Accuracy: 90.62%
[6,000] Accuracy: 84.40%
[7,000] Accuracy: 81.18%
[8,000] Accuracy: 79.23%
[9,000] Accuracy: 78.06%
[10,000] Accuracy: 76.93%
[11,000] Accuracy: 75.13%
[12,000] Accuracy: 74.31%
[13,000] Accuracy: 74.85%
[14,000] Accuracy: 75.30%
[15,000] Accuracy: 75.86%
[16,000] Accuracy: 74.80%
[17,000] Accuracy: 74.85%
[18,000] Accuracy: 75.14%
[19,000] Accuracy: 75.41%
[20,000] Accuracy: 75.61%


Accuracy: 75.61%

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) Quantia Consulting, srl. All rights reserved.